In [48]:
import pandas as pd
import os.path

In [119]:
df = pd.read_csv('data/dataset_final_train_new.csv', sep=',', index_col=0)

In [120]:
df.head()

,name,link,path,file,line,tmm,has_tmm,train_tmm,test_tmm,lm,...,test_lc,dc,has_dc,train_dc,test_dc,lpl,has_lpl,train_lpl,test_lpl,rule
0,hidroh/materialistic,https://github.com/hidroh/materialistic,repositories/hidroh_materialistic,/home/matheus/smell-and-machine-learning/repos...,81,1,1,1,0,1,...,0,1,0,1,0,1,0,1,0,TooManyMethods
1,hidroh/materialistic,https://github.com/hidroh/materialistic,repositories/hidroh_materialistic,/home/matheus/smell-and-machine-learning/repos...,60,1,1,1,0,1,...,0,1,0,1,0,1,0,1,0,TooManyMethods
2,hidroh/materialistic,https://github.com/hidroh/materialistic,repositories/hidroh_materialistic,/home/matheus/smell-and-machine-learning/repos...,36,1,1,1,0,1,...,0,1,0,1,0,1,0,1,0,TooManyMethods
3,hidroh/materialistic,https://github.com/hidroh/materialistic,repositories/hidroh_materialistic,/home/matheus/smell-and-machine-learning/repos...,50,1,1,1,0,1,...,0,1,0,1,0,1,0,1,0,TooManyMethods
4,hidroh/materialistic,https://github.com/hidroh/materialistic,repositories/hidroh_materialistic,/home/matheus/smell-and-machine-learning/repos...,42,1,1,1,0,1,...,0,1,0,1,0,1,0,1,0,TooManyMethods


In [121]:
df[df['lpl'] == 1]['train_lpl'].value_counts()

1    354
0    122
Name: train_lpl, dtype: int64

In [122]:
# def prepare_dataframes(df, label):
#     new_df = df[['file', 'line', label, 'train_{}'.format(label)]].copy()
#     new_df = new_df[new_df[label] == 1]
#     new_df.drop(columns=label, inplace=True)
#     new_df.rename(columns={'train_{}'.format(label): 'train'}, inplace=True)
#     return new_df

# tmm_data = prepare_dataframes(df, 'tmm')
# lm_data = prepare_dataframes(df, 'lm')
# lc_data = prepare_dataframes(df, 'lc')
# dc_data = prepare_dataframes(df, 'dc')
# lpl_data = prepare_dataframes(df, 'lpl')

In [123]:
def prepare_dataframes(df, label):
    new_df = df[['file', 'line',label, 'has_{}'.format(label), 'train_{}'.format(label), 'test_{}'.format(label)]].copy()
    
    #se fazer parte do conjunto de treino e teste daquele smell
    new_df = new_df[new_df[format(label)] == 1]
    new_df.drop(columns=label, inplace=True)
    new_df.rename(columns={'train_{}'.format(label): 'train'}, inplace=True)
    new_df.rename(columns={'test_{}'.format(label): 'test'}, inplace=True)
    new_df.rename(columns={'has_{}'.format(label): 'smell'}, inplace=True) #verifica se tem smell
    return new_df

tmm_data = prepare_dataframes(df, 'tmm')
lm_data = prepare_dataframes(df, 'lm')
lc_data = prepare_dataframes(df, 'lc')
dc_data = prepare_dataframes(df, 'dc')
lpl_data = prepare_dataframes(df, 'lpl')

In [124]:
print(lpl_data.shape)
lpl_data.head()

(476, 5)


,file,line,smell,train,test
0,/home/matheus/smell-and-machine-learning/repos...,81,0,1,0
1,/home/matheus/smell-and-machine-learning/repos...,60,0,1,0
2,/home/matheus/smell-and-machine-learning/repos...,36,0,1,0
3,/home/matheus/smell-and-machine-learning/repos...,50,0,1,0
4,/home/matheus/smell-and-machine-learning/repos...,42,0,1,0


In [125]:
def prepare_path(path):
    return path.replace('/home/matheus/smell-and-machine-learning/', 'data/')

In [126]:
def path_ast_convert(path):
    return path.replace('/repositories/','/ast_results/')

In [127]:
def data_generate(df):
    train = {'content' : [], 'smells' : []}
    test = {'content' : [], 'smells' : []}

    for index, row in df.iterrows():
        #file_path = prepare_path(row['file'])#
        file_path = path_ast_convert(row['file'])
        if os.path.exists(file_path):
            with open(file_path, 'r') as file_content:
    #             content = file_content.readlines()

                content = file_content.readlines()
                if row['train'] == 1:
                    train['content'].append(content)
                    train['smells'].append(row['smell'])
                else:
                    test['content'].append(content)
                    test['smells'].append(row['smell'])
    return list([train,test])

In [128]:
def save_csv(df,label):
    
    data = data_generate(df)
    df_train = pd.DataFrame(data[0])
    df_test = pd.DataFrame(data[1])
    df_train.to_csv('data/df/train_{}.csv'.format(label))
    df_test.to_csv('data/df/test_{}.csv'.format(label))


In [129]:
save_csv(tmm_data,'tmm')
save_csv(lm_data,'lm')
save_csv(lc_data,'lc')
save_csv(dc_data,'dc')
save_csv(lpl_data,'lpl')

In [131]:
data = pd.read_csv('data/df/train_tmm.csv')

In [133]:
data.head()

,Unnamed: 0,content,smells
0,0,"['// isComment\n', 'package io.github.hidroh.m...",1
1,1,"['// isComment\n', 'package io.github.hidroh.m...",1
2,2,"['// isComment\n', 'package io.github.hidroh.m...",1
3,3,"['// isComment\n', 'package io.github.hidroh.m...",1
4,4,"['// isComment\n', 'package io.github.hidroh.m...",1
